In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [2]:
df_green = spark.read.parquet('./data/green_tripdata_2021-01')

In [3]:
df_green.registerTempTable('green')

d:\data-engineering-zoomcamp\.venv\lib\site-packages\pyspark\sql\dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [4]:
df_green_revenue = spark.sql("""
SELECT
    date_trunc('hour', lpep_pickup_datetime) AS hour,
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM green
WHERE 
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [5]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2021-01-30 15:00:00|  41|            164.56|            11|
|2021-01-16 15:00:00|  42|             91.41|             8|
|2021-01-08 13:00:00|  21|             47.47|             2|
|2021-01-05 14:00:00| 145|              39.6|            12|
|2021-01-30 10:00:00| 226|61.099999999999994|             2|
|2021-01-14 07:00:00| 244|             50.69|             3|
|2021-01-19 13:00:00|  65|            108.29|             7|
|2021-01-27 15:00:00|  41| 98.46000000000001|             7|
|2021-01-20 13:00:00| 226|               8.8|             1|
|2021-01-31 10:00:00| 193|              8.16|             1|
|2021-01-22 18:00:00|  74|            300.06|            18|
|2021-01-08 14:00:00| 116|              56.5|             5|
|2021-01-19 10:00:00|  41|167.01999999999998|            10|
|2021-01-14 05:00:00|  8

In [6]:
df_green_revenue.repartition(20).write.parquet('./data/report/revenue/green', mode='overwrite')

In [7]:
df_yellow = spark.read.parquet('./data/yellow_tripdata_2021-01')
df_yellow.registerTempTable('yellow')

d:\data-engineering-zoomcamp\.venv\lib\site-packages\pyspark\sql\dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [8]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [9]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')